In [0]:
pip install snowflake-connector-python

Python interpreter will be restarted.
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Not uninstalling cryptography at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b4ec2569-8b99-4435-9327-d0a392b163b8
    Can't uninstall 'cryptography'. No files were found to uninstall.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b4ec2569-8b99-4435-9327-d0a392b163b8
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
# query:1

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

# Create a SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
spark = SparkSession.builder \
.appName("Customer Total Return Analysis") \
.getOrCreate()

# Read data from Snowflake into Spark DataFrame
store_returns_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_returns").load()
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()
store_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store").load()
customer_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer").load()

# Perform necessary transformations
joined_data = store_returns_df.join(date_dim_df, store_returns_df["SR_RETURNED_DATE_SK"] == date_dim_df["D_DATE_SK"]) \
.filter(date_dim_df["D_YEAR"] == 2000) \
.select("SR_CUSTOMER_SK", "SR_STORE_SK", "SR_RETURN_AMT")
 
customer_total_return = joined_data.groupBy("SR_CUSTOMER_SK", "SR_STORE_SK") \
.sum("SR_RETURN_AMT") \
.withColumnRenamed("SR_CUSTOMER_SK", "CTR_CUSTOMER_SK") \
.withColumnRenamed("SR_STORE_SK", "CTR_STORE_SK") \
.withColumnRenamed("sum(SR_RETURN_AMT)", "CTR_TOTAL_RETURN")
 
avg_return_per_store = customer_total_return.groupBy("CTR_STORE_SK") \
.agg((avg("CTR_TOTAL_RETURN") * 1.2).alias("AVG_TOTAL_RETURN"))

# Filter customers based on the condition
filtered_customers = customer_total_return.join(store_df, store_df["S_STORE_SK"] == customer_total_return["CTR_STORE_SK"]) \
.join(customer_df, customer_df["C_CUSTOMER_SK"] == customer_total_return["CTR_CUSTOMER_SK"]) \
.join(avg_return_per_store, avg_return_per_store["CTR_STORE_SK"] == customer_total_return["CTR_STORE_SK"]) \
.filter(customer_total_return["CTR_TOTAL_RETURN"] > avg_return_per_store["AVG_TOTAL_RETURN"]) \
.filter(store_df["S_STATE"] == "TN") \
.select("C_CUSTOMER_ID") \
.orderBy("C_CUSTOMER_ID") 

try:
    filtered_customers.show(n=100, truncate=False) # Adjust 'n' to the desired number of rows
except Exception as e:
    print("An error occurred while showing the result:", e)

# Apply the filter and count the rows
filtered_customers_count = filtered_customers.distinct().count()

# Print the count of filtered customers
print("Number of filtered customers:", filtered_customers_count)


# filter of record
filtered_df = filtered_customers.filter(filtered_customers["C_CUSTOMER_ID"] == "AAAAAAAAAAAAAEBA")

# Show filtered DataFrame
filtered_df.show()


+----------------+
|C_CUSTOMER_ID   |
+----------------+
|AAAAAAAAAAAAAEBA|
|AAAAAAAAAAAAAJAA|
|AAAAAAAAAAAAALDA|
|AAAAAAAAAAAAAMDA|
|AAAAAAAAAAAABHCA|
|AAAAAAAAAAAACBBA|
|AAAAAAAAAAAACBCA|
|AAAAAAAAAAAACCAA|
|AAAAAAAAAAAACCBA|
|AAAAAAAAAAAACEBA|
|AAAAAAAAAAAACGCA|
|AAAAAAAAAAAACLCA|
|AAAAAAAAAAAACPCA|
|AAAAAAAAAAAACPCA|
|AAAAAAAAAAAADBAA|
|AAAAAAAAAAAADDBA|
|AAAAAAAAAAAADFDA|
|AAAAAAAAAAAADHDA|
|AAAAAAAAAAAADKAA|
|AAAAAAAAAAAADKDA|
|AAAAAAAAAAAAEECA|
|AAAAAAAAAAAAEKBA|
|AAAAAAAAAAAAELAA|
|AAAAAAAAAAAAEPBA|
|AAAAAAAAAAAAFBCA|
|AAAAAAAAAAAAFCCA|
|AAAAAAAAAAAAFEAA|
|AAAAAAAAAAAAFEBA|
|AAAAAAAAAAAAFEBA|
|AAAAAAAAAAAAFIDA|
|AAAAAAAAAAAAFKBA|
|AAAAAAAAAAAAFOBA|
|AAAAAAAAAAAAGEBA|
|AAAAAAAAAAAAGEBA|
|AAAAAAAAAAAAHDCA|
|AAAAAAAAAAAAHJBA|
|AAAAAAAAAAAAHJDA|
|AAAAAAAAAAAAIABA|
|AAAAAAAAAAAAIDAA|
|AAAAAAAAAAAAINDA|
|AAAAAAAAAAAAJABA|
|AAAAAAAAAAAAJBBA|
|AAAAAAAAAAAAJCAA|
|AAAAAAAAAAAAJDAA|
|AAAAAAAAAAAAJEAA|
|AAAAAAAAAAAAJHBA|
|AAAAAAAAAAAAJHDA|
|AAAAAAAAAAAAJJBA|
|AAAAAAAAAAAAJKBA|
|AAAAAAAAAAA

In [0]:
# 2.query:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum, when, count
 
# Create a SparkSession
spark = SparkSession.builder.appName("Weekly Sales Increase Analysis").getOrCreate()
 
# Read web_sales and catalog_sales into DataFrame
web_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "web_sales").load()
catalog_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "catalog_sales").load()
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()
 
# Combine web_sales and catalog_sales into a single DataFrame with common columns
wscs_df = web_sales_df.select("WS_SOLD_DATE_SK", "WS_EXT_SALES_PRICE").unionAll(catalog_sales_df.select("CS_SOLD_DATE_SK", "CS_EXT_SALES_PRICE")).withColumnRenamed("WS_SOLD_DATE_SK", "sold_date_sk").withColumnRenamed("WS_EXT_SALES_PRICE", "sales_price")
 
# Join with date_dim DataFrame to get day name and week sequence
wswscs_df = wscs_df.join(date_dim_df.alias("date_dim_join"), wscs_df["sold_date_sk"] == col("date_dim_join.D_DATE_SK")) \
           .groupBy("date_dim_join.D_WEEK_SEQ") \
           .agg(
               round(sum(when(col("date_dim_join.D_DAY_NAME") == "Sunday", col("sales_price")).otherwise(0)), 2).alias("sun_sales"),
               round(sum(when(col("date_dim_join.D_DAY_NAME") == "Monday", col("sales_price")).otherwise(0)), 2).alias("mon_sales"),
               round(sum(when(col("date_dim_join.D_DAY_NAME") == "Tuesday", col("sales_price")).otherwise(0)), 2).alias("tue_sales"),
               round(sum(when(col("date_dim_join.D_DAY_NAME") == "Wednesday", col("sales_price")).otherwise(0)), 2).alias("wed_sales"),
               round(sum(when(col("date_dim_join.D_DAY_NAME") == "Thursday", col("sales_price")).otherwise(0)), 2).alias("thu_sales"),
               round(sum(when(col("date_dim_join.D_DAY_NAME") == "Friday", col("sales_price")).otherwise(0)), 2).alias("fri_sales"),
               round(sum(when(col("date_dim_join.D_DAY_NAME") == "Saturday", col("sales_price")).otherwise(0)), 2).alias("sat_sales")
           )
 
# Selecting data for year 2001 and 2002
data_2001 = wswscs_df.join(date_dim_df.alias("date_dim_2001"), wswscs_df["D_WEEK_SEQ"] == col("date_dim_2001.d_week_seq")) \
    .where(col("date_dim_2001.d_year") == 2001) \
    .selectExpr("date_dim_2001.d_week_seq as d_week_seq1", "sun_sales as sun_sales1", "mon_sales as mon_sales1",
                "tue_sales as tue_sales1", "wed_sales as wed_sales1", "thu_sales as thu_sales1",
                "fri_sales as fri_sales1", "sat_sales as sat_sales1")
 
data_2002 = wswscs_df.join(date_dim_df.alias("date_dim_2002"), wswscs_df["D_WEEK_SEQ"] == col("date_dim_2002.d_week_seq")) \
    .where(col("date_dim_2002.d_year") == 2002) \
    .selectExpr("date_dim_2002.d_week_seq as d_week_seq2", "sun_sales as sun_sales2", "mon_sales as mon_sales2",
                "tue_sales as tue_sales2", "wed_sales as wed_sales2", "thu_sales as thu_sales2",
                "fri_sales as fri_sales2", "sat_sales as sat_sales2")
 
# Joining data for the two years
final_data = data_2001.join(data_2002, (data_2001["d_week_seq1"] == data_2002["d_week_seq2"] - 53)) \
    .orderBy("d_week_seq1") \
    .selectExpr("d_week_seq1",
                "round(sun_sales1/sun_sales2, 2) as sun_sales_ratio",
                "round(mon_sales1/mon_sales2, 2) as mon_sales_ratio",
                "round(tue_sales1/tue_sales2, 2) as tue_sales_ratio",
                "round(wed_sales1/wed_sales2, 2) as wed_sales_ratio",
                "round(thu_sales1/thu_sales2, 2) as thu_sales_ratio",
                "round(fri_sales1/fri_sales2, 2) as fri_sales_ratio",
                "round(sat_sales1/sat_sales2, 2) as sat_sales_ratio")
 
# Displaying the final result
final_data.show()
total_count = final_data.select(count("d_week_seq1"))
total_count.show()

filtered_df = final_data.filter(final_data["d_week_seq1"] == 5270)

# Show filtered DataFrame
filtered_df.show()


+-----------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|d_week_seq1|sun_sales_ratio|mon_sales_ratio|tue_sales_ratio|wed_sales_ratio|thu_sales_ratio|fri_sales_ratio|sat_sales_ratio|
+-----------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|       5283|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|
|       5283|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|
|       5283|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|
|       5283|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|
|       5283|           1.00|           1.00|           1.00|           1.00|           1.00|           1.00|         

In [0]:
# query: 3

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.functions import count

# Create SparkSession
spark = SparkSession.builder \
    .appName("Convert SQL to PySpark") \
    .getOrCreate()

# Read tables as DataFrames
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
item_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "item").load()

# Join DataFrames and apply filters
joined_df = date_dim_df.join(store_sales_df, date_dim_df["d_date_sk"] == store_sales_df["ss_sold_date_sk"]) \
    .join(item_df, store_sales_df["ss_item_sk"] == item_df["i_item_sk"]) \
    .where((date_dim_df["d_moy"] == 11) & (item_df["i_manufact_id"] == 128))

# Perform aggregation
agg_df = joined_df.groupBy(date_dim_df["d_year"], item_df["i_brand_id"], item_df["i_brand"]) \
    .agg(spark_sum("ss_ext_sales_price").alias("sum_agg"))

# Order by specified columns
ordered_df = agg_df.orderBy("d_year", "sum_agg", "i_brand_id", ascending=[True, False, True])

# Show DataFrame
ordered_df.show()
# sum of all records
sum_df = ordered_df.select(sum('sum_agg'))

# Show DataFrame
sum_df.show()

# count of records
total_count = ordered_df.select(count("sum_agg"))
total_count.show()

# filter of record
filtered_df = ordered_df.filter(ordered_df["i_brand_id"] == 3004001)

# Show filtered DataFrame
filtered_df.show()



+------+----------+--------------------+----------+
|d_year|i_brand_id|             i_brand|   sum_agg|
+------+----------+--------------------+----------+
|  1998|   8015003|      scholarmaxi #3|8109206.01|
|  1998|   2001002|     amalgimporto #2|8097233.52|
|  1998|  10006017|   corpunivamalg #17|8087166.91|
|  1998|   6012005|     importobrand #5|8084274.20|
|  1998|  10003017|exportiunivamalg #17|8082571.61|
|  1998|   7014006| edu packnameless #6|8077215.29|
|  1998|   6007002|        brandcorp #2|8070730.80|
|  1998|   8013003|      exportimaxi #3|8067421.37|
|  1998|   7014007| edu packnameless #7|8063499.38|
|  1998|  10009013|   maxiunivamalg #13|8054987.47|
|  1998|  10016003|   corpamalgamalg #3|8047113.41|
|  1998|  10011006|  amalgamalgamalg #6|8045093.69|
|  1998|   8003010| exportinameless #10|8041132.21|
|  1998|   8005003|  scholarnameless #3|8035518.75|
|  1998|  10008002|namelessunivamalg #2|8035242.64|
|  1998|   8007001|    brandnameless #1|8029976.18|
|  1998|   9

In [0]:
#query: 4
sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, when, count
 
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Customer Spending Analysis") \
    .getOrCreate()
 
# Read data from Snowflake into Spark DataFrame
customer_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer").load()
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
catalog_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "catalog_sales").load()
 
# Define the qualification substitution parameters
YEAR = 2001
 
# Calculate year total for store sales
from pyspark.sql.functions import sum, col, lit
 
# Calculate year total for store sales
store_sales_year_total = store_sales_df.groupBy("ss_customer_sk").agg(
    sum(((store_sales_df["ss_ext_list_price"] - store_sales_df["ss_ext_wholesale_cost"] - store_sales_df["ss_ext_discount_amt"]) + store_sales_df["ss_ext_sales_price"]) / 2).alias("year_total")
).withColumn("sale_type", lit("s"))
 
# Calculate year total for catalog sales
catalog_sales_year_total = catalog_sales_df.groupBy("cs_bill_customer_sk").agg(
    sum((((catalog_sales_df["cs_ext_list_price"] - catalog_sales_df["cs_ext_wholesale_cost"] - catalog_sales_df["cs_ext_discount_amt"]) + catalog_sales_df["cs_ext_sales_price"]) / 2)).alias("year_total")
).withColumn("sale_type", lit("c"))
 
 
# Calculate year total for catalog sales
catalog_sales_year_total = catalog_sales_df.groupBy("cs_bill_customer_sk").agg(
    sum((((catalog_sales_df["cs_ext_list_price"] - catalog_sales_df["cs_ext_wholesale_cost"] - catalog_sales_df["cs_ext_discount_amt"]) + catalog_sales_df["cs_ext_sales_price"]) / 2)).alias("year_total")
).withColumn("sale_type", lit("c"))
 
# Find customers who spent more via catalog than in stores
preferred_customers = store_sales_year_total.join(catalog_sales_year_total, store_sales_year_total["ss_customer_sk"] == catalog_sales_year_total["cs_bill_customer_sk"], "inner") \
    .join(customer_df, store_sales_year_total["ss_customer_sk"] == customer_df["c_customer_sk"], "inner") \
    .filter((store_sales_year_total["year_total"] > 0) & (catalog_sales_year_total["year_total"] > 0) & \
            (catalog_sales_year_total["year_total"] / store_sales_year_total["year_total"] > 1)) \
    .select(
        customer_df["c_customer_id"].alias("customer_id"),
        customer_df["c_first_name"].alias("customer_first_name"),
        customer_df["c_last_name"].alias("customer_last_name"),
        customer_df["c_birth_country"].alias("customer_birth_country"),
        customer_df["c_preferred_cust_flag"].alias("customer_preferred_cust_flag")
    )
 
# Show the result
preferred_customers.show()
print('customer_id count:')
total_count = preferred_customers.select(count("customer_id"))
total_count.show()

+----------------+-------------------+------------------+----------------------+----------------------------+
|     customer_id|customer_first_name|customer_last_name|customer_birth_country|customer_preferred_cust_flag|
+----------------+-------------------+------------------+----------------------+----------------------------+
|AAAAAAAAJBLMOCAA|            William|          Chandler|                MONACO|                           Y|
|AAAAAAAAADFDLGDA|            Brandon|         Dominguez|               ALBANIA|                           Y|
|AAAAAAAADGOJEFDA|             Samual|             Adams|              CAMBODIA|                           N|
|AAAAAAAAFCJAGHCA|                Lee|              null|                  null|                        null|
|AAAAAAAANBDJMADA|             Bonnie|           Shelton|         FRENCH GUIANA|                           Y|
|AAAAAAAADNGALCBA|               Nona|              null|                GREECE|                           Y|
|AAAAAAAAJ

In [0]:
# 5. query

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum, col, when, lit, date_add

# Create SparkSession
spark = SparkSession.builder \
    .appName("Convert SQL to PySpark") \
    .getOrCreate()

# Read tables as DataFrames
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
store_returns_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_returns").load()
catalog_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "catalog_sales").load()
catalog_returns_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "catalog_returns").load()
web_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "web_sales").load()
web_returns_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "web_returns").load()
store_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store").load()
catalog_page_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "catalog_page").load()
web_site_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "web_site").load()
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()

# Create temporary views for tables
store_sales_df.createOrReplaceTempView("store_sales")
store_returns_df.createOrReplaceTempView("store_returns")
catalog_sales_df.createOrReplaceTempView("catalog_sales")
catalog_returns_df.createOrReplaceTempView("catalog_returns")
web_sales_df.createOrReplaceTempView("web_sales")
web_returns_df.createOrReplaceTempView("web_returns")
store_df.createOrReplaceTempView("store")
catalog_page_df.createOrReplaceTempView("catalog_page")
web_site_df.createOrReplaceTempView("web_site")
date_dim_df.createOrReplaceTempView("date_dim")

# Execute the PySpark code
result_df = store_sales_df.join(

    date_dim_df,
    store_sales_df["ss_sold_date_sk"] == date_dim_df["d_date_sk"]
).join(
    store_df,
    store_sales_df["ss_store_sk"] == store_df["s_store_sk"]
).where(
    (col("d_date").between("2000-08-23", date_add("2000-08-23", 14))) &
    (col("store_sales_df") == "store")
).groupBy(
    "s_store_id"
).agg(
    spark_sum("ss_sales_price").alias("sales"),
    spark_sum("ss_net_profit").alias("profit"),
    spark_sum(when(col("sr_return_amt").isNull(), 0).otherwise(col("sr_return_amt"))).alias("returns"),
    spark_sum(when(col("sr_net_loss").isNull(), 0).otherwise(col("sr_net_loss"))).alias("profit_loss")
).select(
    col("channel"),
    col("id"),
    spark_sum("sales").alias("sales"),
    spark_sum("returns").alias("returns"),
    (spark_sum("profit") - spark_sum("profit_loss")).alias("profit")
).groupBy(
    rollup("channel", "id")
).orderBy(
    "channel",
    "id"
)

# Show DataFrame
result_df.show()

In [0]:
#6.query

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count
 
# Create SparkSession
spark = SparkSession.builder \
    .appName("Convert SQL to PySpark") \
    .getOrCreate()
 
# Read necessary tables
customer_address_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer_address").load()
customer_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer").load()
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()
item_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "item").load()
 
# Filter date_dim for the specified year and month
date_dim_filtered_df = date_dim_df.filter((col("d_year") == 2001) & (col("d_moy") == 1))
 
# Join the tables and perform necessary operations
result_df = customer_address_df.alias("a") \
    .join(customer_df.alias("c"), col("a.ca_address_sk") == col("c.c_current_addr_sk")) \
    .join(store_sales_df.alias("s"), col("c.c_customer_sk") == col("s.ss_customer_sk")) \
    .join(date_dim_filtered_df.alias("d"), col("s.ss_sold_date_sk") == col("d.d_date_sk")) \
    .join(item_df.alias("i"), col("s.ss_item_sk") == col("i.i_item_sk")) \
    .groupBy("a.ca_state") \
    .agg({"*": "count"}) \
    .withColumnRenamed("count(1)", "cnt") \
    .filter(col("cnt") >= 10) \
    .orderBy("cnt", "a.ca_state")
 
# Show the result
result_df.show()

total_count = result_df.select(count("sum_agg"))
total_count.show()



+--------+-------+
|ca_state|    cnt|
+--------+-------+
|      DC|  88676|
|      DE| 261918|
|      HI| 353226|
|      RI| 440450|
|      CT| 707140|
|      NH| 907516|
|      MA|1234544|
|      SC|4094370|
|      ND|4688950|
|      LA|4866379|
|      WV|4885452|
|      MT|4970962|
|      CA|5160869|
|      IN|8054158|
|      NE|8263964|
|      TN|8460637|
|    null|8526207|
|      IA|8788940|
|      NC|8895997|
|      IL|8953018|
+--------+-------+
only showing top 20 rows



In [0]:
#query: 7

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col
 
# Create SparkSession
spark = SparkSession.builder \
    .appName("Convert SQL to PySpark") \
    .getOrCreate()
 
# Read necessary tables
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
customer_demographics_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer_demographics").load()
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()
item_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "item").load()
promotion_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "promotion").load()
 
# Filter date_dim for the specified year
date_dim_filtered_df = date_dim_df.filter(col("d_year") == 2000)
 
# Join the tables and perform necessary operations
result_df = store_sales_df.alias("ss") \
    .join(customer_demographics_df.alias("cd"), col("ss.ss_cdemo_sk") == col("cd.cd_demo_sk")) \
    .join(date_dim_filtered_df.alias("d"), col("ss.ss_sold_date_sk") == col("d.d_date_sk")) \
    .join(item_df.alias("i"), col("ss.ss_item_sk") == col("i.i_item_sk")) \
    .join(promotion_df.alias("p"), col("ss.ss_promo_sk") == col("p.p_promo_sk")) \
    .filter((col("cd.cd_gender") == "M") &
            (col("cd.cd_marital_status") == "S") &
            (col("cd.cd_education_status") == "College") &
            ((col("p.p_channel_email") == "N") | (col("p.p_channel_event") == "N"))) \
    .groupBy("i.i_item_id") \
    .agg(avg("ss.ss_quantity").alias("agg1"),
         avg("ss.ss_list_price").alias("agg2"),
         avg("ss.ss_coupon_amt").alias("agg3"),
         avg("ss.ss_sales_price").alias("agg4")) \
    .orderBy("i.i_item_id")
 
# Show the result
result_df.show()

+----------------+---------+---------+----------+---------+
|       i_item_id|     agg1|     agg2|      agg3|     agg4|
+----------------+---------+---------+----------+---------+
|AAAAAAAAHPBJFAAA|50.200000|74.191231|223.677914|37.163957|
|AAAAAAAAHPBKBAAA|44.585784|71.127878|139.085196|41.612083|
|AAAAAAAAHPBKEAAA|50.542320|78.371781|218.257406|40.271223|
|AAAAAAAAHPBLAAAA|47.623306|68.934278|100.518747|33.447711|
|AAAAAAAAHPBLDAAA|49.862069|73.515252|146.751379|36.030849|
|AAAAAAAAHPBMCAAA|52.853583|75.399875|182.279190|42.183323|
|AAAAAAAAHPBMFAAA|51.182927|81.978927|169.831127|40.875951|
|AAAAAAAAHPBNBAAA|46.204188|82.177192|200.035354|42.832068|
|AAAAAAAAHPBNEAAA|53.287834|78.194911|266.799021|38.547567|
|AAAAAAAAHPBOAAAA|53.097619|83.946000|107.135976|38.936524|
|AAAAAAAAHPBODAAA|55.334975|70.992868|130.340983|32.699828|
|AAAAAAAAHPBPCAAA|46.836735|79.985587|169.134987|33.586412|
|AAAAAAAAHPBPFAAA|49.193460|72.728747|168.208184|36.687398|
|AAAAAAAAHPCABAAA|56.095142|73.083144|15

In [0]:
# query: 8

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring, sum as _sum
 
# Initialize Spark session
spark = SparkSession.builder \
.appName("Snowflake to Spark") \
.getOrCreate()
 
# Read data from Snowflake into Spark DataFrames
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()
store_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store").load()
customer_address_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer_address").load()
customer_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer").load()
 
# Subquery to filter the zip codes
valid_zips_df = customer_address_df.select(substring("CA_ZIP", 1, 5).alias("CA_ZIP")) \
.filter(col("CA_ZIP").isin(
'67436', '26121', '38443', '63157', '68856', '19485', '86425', '26741',
'70991', '60899', '63573', '47556', '56193', '93314', '87827', '62017',
'85067', '95390', '48091', '10261', '81845', '41790', '42853', '24675',
'12840', '60065', '84430', '57451', '24021', '91735', '75335', '71935',
'34482', '56943', '70695', '52147', '56251', '28411', '86653', '23005',
'22478', '29031', '34398', '15365', '42460', '33337', '59433', '73943',
'72477', '74081', '74430', '64605', '39006', '11226', '49057', '97308',
'42663', '18187', '19768', '43454', '32147', '76637', '51975', '11181',
'45630', '33129', '45995', '64386', '55522', '26697', '20963', '35154',
'64587', '49752', '66386', '30586', '59286', '13177', '66646', '84195',
'74316', '36853', '32927', '12469', '11904', '36269', '17724', '55346',
'12595', '53988', '65439', '28015', '63268', '73590', '29216', '82575',
'69267', '13805', '91678', '79460', '94152', '14961', '15419', '48277',
'62588', '55493', '28360', '14152', '55225', '18007', '53705', '56573',
'80245', '71769', '57348', '36845', '13039', '17270', '22363', '83474',
'25294', '43269', '77666', '15488', '99146', '64441', '43338', '38736',
'62754', '48556', '86057', '23090', '38114', '66061', '18910', '84385',
'23600', '19975', '27883', '65719', '19933', '32085', '49731', '40473',
'27190', '46192', '23949', '44738', '12436', '64794', '68741', '15333',
'24282', '49085', '31844', '71156', '48441', '17100', '98207', '44982',
'20277', '71496', '96299', '37583', '22206', '89174', '30589', '61924',
'53079', '10976', '13104', '42794', '54772', '15809', '56434', '39975',
'13874', '30753', '77598', '78229', '59478', '12345', '55547', '57422',
'42600', '79444', '29074', '29752', '21676', '32096', '43044', '39383',
'37296', '36295', '63077', '16572', '31275', '18701', '40197', '48242',
'27219', '49865', '84175', '30446', '25165', '13807', '72142', '70499',
'70464', '71429', '18111', '70857', '29545', '36425', '52706', '36194',
'42963', '75068', '47921', '74763', '90990', '89456', '62073', '88397',
'73963', '75885', '62657', '12530', '81146', '57434', '25099', '41429',
'98441', '48713', '52552', '31667', '14072', '13903', '44709', '85429',
'58017', '38295', '44875', '73541', '30091', '12707', '23762', '62258',
'33247', '78722', '77431', '14510', '35656', '72428', '92082', '35267',
'43759', '24354', '90952', '11512', '21242', '22579', '56114', '32339',
'52282', '41791', '24484', '95020', '28408', '99710', '11899', '43344',
'72915', '27644', '62708', '74479', '17177', '32619', '12351', '91339',
'31169', '57081', '53522', '16712', '34419', '71779', '44187', '46206',
'96099', '61910', '53664', '12295', '31837', '33096', '10813', '63048',
'31732', '79118', '73084', '72783', '84952', '46965', '77956', '39815',
'32311', '75329', '48156', '30826', '49661', '13736', '92076', '74865',
'88149', '92397', '52777', '68453', '32012', '21222', '52721', '24626',
'18210', '42177', '91791', '75251', '82075', '44372', '45542', '20609',
'60115', '17362', '22750', '90434', '31852', '54071', '33762', '14705',
'40718', '56433', '30996', '40657', '49056', '23585', '66455', '41021',
'74736', '72151', '37007', '21729', '60177', '84558', '59027', '93855',
'60022', '86443', '19541', '86886', '30532', '39062', '48532', '34713',
'52077', '22564', '64638', '15273', '31677', '36138', '62367', '60261',
'80213', '42818', '25113', '72378', '69802', '69096', '55443', '28820',
'13848', '78258', '37490', '30556', '77380', '28447', '44550', '26791',
'70609', '82182', '33306', '43224', '22322', '86959', '68519', '14308',
'46501', '81131', '34056', '61991', '19896', '87804', '65774', '92564'
))
 
# Second subquery to filter zip codes with preferred customers
preferred_cust_zips_df = customer_df.filter(col("C_CURRENT_HH_DEMOGRAPHIC") == "preferred") \
.join(customer_address_df, customer_df.C_CURRENT_ADDR_SK == customer_address_df.CA_ADDRESS_SK) \
.select(substring(customer_address_df.CA_ZIP, 1, 5).alias("CA_ZIP"))
 
# Perform the intersection
common_zips_df = valid_zips_df.select("CA_ZIP").intersect(preferred_cust_zips_df.select("CA_ZIP"))
 
# Join the DataFrames to filter sales by these zip codes
result_df = store_sales_df.join(common_zips_df, store_sales_df.SS_ADDR_SK == customer_address_df.CA_ADDRESS_SK) \
.join(date_dim_df, store_sales_df.SS_SOLD_DATE_SK == date_dim_df.D_DATE_SK) \
.join(store_df, store_sales_df.SS_STORE_SK == store_df.S_STORE_SK) \
.groupBy(store_df.S_STORE_NAME) \
.agg(_sum(store_sales_df.SS_SALES_PRICE).alias("TOTAL_SALES"))
result_df.show()

In [0]:
# query: 9

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}


from pyspark.sql.functions import col, avg, count, when
 
# Initialize Spark session
spark = SparkSession.builder \
.appName("SQL to PySpark Conversion") \
.getOrCreate()
 
# Read data from Snowflake into Spark DataFrames
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
reason_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "reason").load()
 
# Function to calculate the metric based on the condition
def calculate_metric(df, lower, upper, threshold):

  count_value = df.filter((col("ss_quantity") >= lower) & (col("ss_quantity") <= upper)).count()
  if count_value > threshold:
    return df.filter((col("ss_quantity") >= lower) & (col("ss_quantity") <= upper)).agg(avg("ss_ext_list_price")).collect()[0][0]
  else:
    return df.filter((col("ss_quantity") >= lower) & (col("ss_quantity") <= upper)).agg(avg("ss_net_profit")).collect()[0][0]
 
# Calculate each bucket value
bucket1 = calculate_metric(store_sales_df, 1, 20, 3672)
bucket2 = calculate_metric(store_sales_df, 21, 40, 3392)
bucket3 = calculate_metric(store_sales_df, 41, 60, 32784)
bucket4 = calculate_metric(store_sales_df, 61, 80, 26032)
bucket5 = calculate_metric(store_sales_df, 81, 100, 23982)
 
# Create a DataFrame with the calculated bucket values
bucket_values = [(bucket1, bucket2, bucket3, bucket4, bucket5)]
buckets_df = spark.createDataFrame(bucket_values, ["bucket1", "bucket2", "bucket3", "bucket4", "bucket5"])
 
# Filter the reason table and join with the buckets DataFrame
result_df = reason_df.filter(col("r_reason_sk") == 1).crossJoin(buckets_df)
result_df.show()

+-----------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|R_REASON_SK|     R_REASON_ID|      R_REASON_DESC|             bucket1|             bucket2|             bucket3|             bucket4|             bucket5|
+-----------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          1|AAAAAAAABAAAAAAA|Package was damaged|795.2997990000000...|2310.257528000000...|3824.761249000000...|5340.264830000000...|6855.546035000000...|
+-----------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [0]:
# query: 10

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql.functions import col, count
 
# Initialize Spark session
spark = SparkSession.builder \
.appName("SQL to PySpark Conversion") \
.getOrCreate()
 
# Read data from Snowflake into Spark DataFrames
customer_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer").load()
customer_address_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer_address").load()
customer_demographics_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "customer_demographics").load()
store_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "store_sales").load()
web_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "web_sales").load()
catalog_sales_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "catalog_sales").load()
date_dim_df = spark.read.format("snowflake").options(**sfOptions).option("dbtable", "date_dim").load()
 
# Filter date_dim for the required year and months
filtered_date_dim_df = date_dim_df.filter((col("d_year") == 2002) & (col("d_moy").between(4, 7)))
 
# Filter store_sales, web_sales, and catalog_sales for the required date range
filtered_store_sales_df = store_sales_df.join(filtered_date_dim_df, store_sales_df["ss_sold_date_sk"] == filtered_date_dim_df["d_date_sk"])
filtered_web_sales_df = web_sales_df.join(filtered_date_dim_df, web_sales_df["ws_sold_date_sk"] == filtered_date_dim_df["d_date_sk"])
filtered_catalog_sales_df = catalog_sales_df.join(filtered_date_dim_df, catalog_sales_df["cs_sold_date_sk"] == filtered_date_dim_df["d_date_sk"])
 
# Filter customers based on store_sales, web_sales, and catalog_sales
store_sales_customers = filtered_store_sales_df.select("ss_customer_sk").distinct()
web_sales_customers = filtered_web_sales_df.select("ws_bill_customer_sk").distinct()
catalog_sales_customers = filtered_catalog_sales_df.select("cs_ship_customer_sk").distinct()
 
# Union all customers
all_customers = store_sales_customers.union(web_sales_customers).union(catalog_sales_customers).distinct()
 
# Filter customer and customer_address tables
filtered_customer_address_df = customer_address_df.filter(col("ca_county").isin(
'Lycoming County', 'Sheridan County', 'Kandiyohi County', 'Pike County', 'Greene County'
))
 
# Join customer with filtered customer_address
customer_with_address = customer_df.join(filtered_customer_address_df, customer_df["c_current_addr_sk"] == filtered_customer_address_df["ca_address_sk"])
 
# Filter customers who are in the sales data
final_customers = customer_with_address.join(all_customers, customer_with_address["c_customer_sk"] == all_customers["ss_customer_sk"])
 
# Join with customer_demographics
final_data = final_customers.join(customer_demographics_df, customer_demographics_df["cd_demo_sk"] == final_customers["c_current_cdemo_sk"])
 
# Group by required columns and aggregate
result_df = final_data.groupBy(
"cd_gender",
"cd_marital_status",
"cd_education_status",
"cd_purchase_estimate",
"cd_credit_rating",
"cd_dep_count",
"cd_dep_employed_count",
"cd_dep_college_count"
).agg(
count("*").alias("cnt1"),
count("*").alias("cnt2"),
count("*").alias("cnt3"),
count("*").alias("cnt4"),
count("*").alias("cnt5"),
count("*").alias("cnt6")
).orderBy(
"cd_gender",
"cd_marital_status",
"cd_education_status",
"cd_purchase_estimate",
"cd_credit_rating",
"cd_dep_count",
"cd_dep_employed_count",
"cd_dep_college_count"
).limit(100)
# Show the result
result_df.show()

+---------+-----------------+-------------------+--------------------+----------------+------------+---------------------+--------------------+----+----+----+----+----+----+
|cd_gender|cd_marital_status|cd_education_status|cd_purchase_estimate|cd_credit_rating|cd_dep_count|cd_dep_employed_count|cd_dep_college_count|cnt1|cnt2|cnt3|cnt4|cnt5|cnt6|
+---------+-----------------+-------------------+--------------------+----------------+------------+---------------------+--------------------+----+----+----+----+----+----+
|        F|                M|    Advanced Degree|                 500|        Low Risk|           5|                    4|                   4|   1|   1|   1|   1|   1|   1|
|        F|                M|    Advanced Degree|                 500|        Low Risk|           5|                    4|                   5|   1|   1|   1|   1|   1|   1|
|        F|                M|    Advanced Degree|                 500|        Low Risk|           5|                    5|        

In [0]:
# 6. query

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

In [0]:
# 6. query

sfOptions = {
  "sfURL" : "https://uh98701.ap-southeast-1.snowflakecomputing.com",
  "sfAccount" : "uh98701.ap-southeast-1",
  "sfUser" : "UMESHCHANDRAKADEM",
  "sfPassword" : "Umesh.K@127",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "TPCDS_SF10TCL",
  "sfWarehouse" : "COMPUTE_WH"
}

from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, avg

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Customer Sales Count") \
    .getOrCreate()

# Read data from the appropriate sources and create DataFrames
customer_address_df = spark.read.table("customer_address").select("ca_address_sk", "ca_state")
customer_df = spark.read.table("customer").select("c_customer_sk", "c_current_addr_sk")
store_sales_df = spark.read.table("store_sales").select("ss_customer_sk", "ss_sold_date_sk", "ss_item_sk")
date_dim_df = spark.read.table("date_dim").select("d_date_sk", "d_month_seq")
item_df = spark.read.table("item").select("i_item_sk", "i_current_price", "i_category")

# Filter date dimension to get the desired month sequence
desired_month_seq = date_dim_df.filter((date_dim_df["d_year"] == 2001) & (date_dim_df["d_moy"] == 1)) \
    .select("d_month_seq").distinct()

# Calculate the average price per category
avg_price_per_category = item_df.join(
    item_df.groupBy("i_category").agg(avg("i_current_price").alias("avg_price_per_category")),
    "i_category"
)

# Join all tables and apply necessary filters
result_df = customer_address_df.join(customer_df, customer_address_df["ca_address_sk"] == customer_df["c_current_addr_sk"]) \
    .join(store_sales_df, customer_df["c_customer_sk"] == store_sales_df["ss_customer_sk"]) \
    .join(date_dim_df, store_sales_df["ss_sold_date_sk"] == date_dim_df["d_date_sk"]) \
    .join(item_df, store_sales_df["ss_item_sk"] == item_df["i_item_sk"]) \
    .join(desired_month_seq, date_dim_df["d_month_seq"] == desired_month_seq["d_month_seq"]) \
    .join(avg_price_per_category, item_df["i_category"] == avg_price_per_category["i_category"]) \
    .where(item_df["i_current_price"] > 1.2 * avg_price_per_category["avg_price_per_category"]) \
    .groupBy(customer_address_df["ca_state"]) \
    .agg(count("*").alias("cnt")) \
    .filter(col("cnt") >= 10) \
    .orderBy(col("cnt"), col("ca_state"))

# Show the result
result_df.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1369089997198724>:22
     17 spark = SparkSession.builder \
     18     .appName("Customer Sales Count") \
     19     .getOrCreate()
     21 # Read data from the appropriate sources and create DataFrames
---> 22 customer_address_df = spark.read.table("customer_address").select("ca_address_sk", "ca_state")
     23 customer_df = spark.read.table("customer").select("c_customer_sk", "c_current_addr_sk")
     24 store_sales_df = spark.read.table("store_sales").select("ss_customer_sk", "ss_sold_date_sk", "ss_item_sk")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter(